In [ ]:
import numpy as np
import xarray as xr

In [ ]:
coordsA = {'A': ['bad', 'good']}
coordsF = {'F': ['unfit', 'fit']}
coordsR = {'R': ['muddy', 'dry']}
coordsT = {'T': ['poor', 'average', 'top']}
coordsQ = {'Q': ['fail', 'qualified']}

pF = xr.DataArray([0.7, 0.3], dims=('F'), coords=coordsF)

pA_F = xr.DataArray([[0.95, 0.05], [0.2, 0.8]], dims=('F', 'A'), coords={**coordsA, **coordsF})

pR = xr.DataArray([0.6, 0.4], dims=('R'), coords=coordsR)
pT_RF = xr.DataArray([[[0.3, 0.4, 0.3], [0.7, 0.25, 0.05]], [[0.02, 0.08, 0.90], [0.2, 0.3, 0.5]]], dims=('F', 'R', 'T'), coords={**coordsT, **coordsR, **coordsF}) # times/groups are in "reverse" order compared to Koller
pQ_T = xr.DataArray([[0.99, 0.01], [0.4, 0.6], [0.1, 0.9]], dims=('T', 'Q'), coords={**coordsT, **coordsQ}) # here it changes the order of the 3 rows

def normalize(a):
    return a / a.sum()

# derived joint probabilities for the exercices
pAF = pF * pA_F
pAFT = (pAF * pR * pT_RF).sum(dim='R')
p_all = pR*pF*pT_RF*pA_F*pQ_T

In [ ]:
pF

In [ ]:
pA_F

In [ ]:
p_all.sum()

# Simple 2 variable case

In [ ]:
pAF # joint distribution

In [ ]:
# Q1
# proba of being fit, sum rule
print("A proportion of", pAF.sum(dim='A').sel(F='fit').values, "is fit")
# but we also have all values p(A)
pAF.sum(dim='A')

In [ ]:
# Q2
# proba of being fit, knowing we did good at the aerobic test
# conditioning (taking just a value) and renormalizing
print("Given A=good, the proba of being fit is", normalize(pAF.sel(A='good')).sel(F='fit').values)
# but we also have all values p(F|A=good)
normalize(pAF.sel(A='good'))

In [ ]:
# exploring
normalize(pAF.sel(A='bad'))

In [ ]:
# more exploration
pAF.sum(dim='F')

# Three variables

In [ ]:
print("shape:", pAFT.shape)
print("total size:", pAFT.size)
print(pAFT.sum().values)

pAFT

In [ ]:
# Q3
# knowing there was a good test result A=good,
# what is the probability of getting a time in the top, T=top?
print("Given A=good, prob(T=top) =", normalize(pAFT.sel(A='good').sum(dim='F')).sel(T='top').values)
# the distribution (on T)
normalize(pAFT.sel(A='good').sum(dim='F'))

In [ ]:
# let's check what we get with a bad test result
normalize(pAFT.sel(A='bad').sum(dim='F'))

In [ ]:
# can we be fit but fail tests and races
normalize(pAFT.sel(A='bad', T='poor')) # ... not so much

In [ ]:
# if we know we're fit, what is the probabily to perform at the race? 
normalize(pAFT.sel(F='fit').sum('A'))

In [ ]:
# same, but knowing a test results
normalize(pAFT.sel(A='bad', F='fit'))
# (no change, so there is a conditional independance)
# (T seems indep of A, given F=fit)
# (already knowing F=fit, knowing A does not bring information on T)

In [ ]:
# same with F='unfit'
normalize(pAFT.sel(A='bad', F='unfit')) / normalize(pAFT.sel(F='unfit').sum('A'))
# ... the ratio p(T|A=bad,F=unfit) / p(T|F=unfit) is 1 for all T

# All variables

In [ ]:
print("shape:", p_all.shape)
print("total size:", p_all.size)
print(p_all.sum().values)

In [ ]:
p_all

In [ ]:
# Q4
# probability of getting qualified, given a good aerobic test (and nothing else)
normalize(p_all.sel(A='good').sum(dim=('R', 'F', 'T')))

In [ ]:
# it is still low, so let's check the probability with a bad test result
normalize(p_all.sel(A='bad').sum(dim=('R', 'F', 'T')))

In [ ]:
# and the "prior" probability with no information
p_all.sum(dim=('A', 'R', 'F', 'T'))

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
θvalues = np.linspace(0, 1, 21)

#pθ = xr.DataArray(np.ones(21)/21, dims=('θ',), coords={"θ": θvalues})
prior = 1-(0.5-θvalues)**2
prior /= np.sum(prior)

plt.plot(θvalues, prior)
plt.show()

x = 1
p_x_θ = θvalues

plt.plot(θvalues, p_x_θ)
plt.show()

#x123 =      (1,         1,         0)
p_x123_θ = θvalues * θvalues * (1-θvalues)

plt.plot(θvalues, p_x123_θ)
plt.show()

p_θ_and_x123 = p_x123_θ * prior
p_θ_x123 = p_θ_and_x123 / np.sum(p_θ_and_x123)

plt.plot(θvalues, p_θ_x123)
plt.show()
